In [2]:
import pandas as pd
import numpy as np

stock_prices = pd.read_csv('/Users/hnunez/Documents/GitHub/Datathon2023/data/stock_prices.csv')
stock_prices['timestamp'] = pd.to_datetime(stock_prices['timestamp'])
stock_prices = stock_prices.set_index('timestamp').sort_index(ascending=True)
shifted_stock_prices = stock_prices.shift(1).add_suffix('_last_close')

column_mapping = {
    'AA': 'AAL',
    'UA': 'UAL',
    'B6': 'JBLU',
    'AS': 'ALK',
    'NK': 'SAVE',
    'WN': 'LUV',
    'DL': 'DAL',
    'HA': 'HA'
}
tickers = ['AAL', 'UAL', 'JBLU', 'ALK', 'SAVE', 'LUV', 'DAL', 'HA']

stock_prices.rename(columns=column_mapping, inplace=True)

# Calculate log prices and add suffix '_log_price'
stock_price_log = stock_prices.apply(np.log).add_suffix('_log_price')

# Define the rolling windows
windows = [7, 14, 30, 90]
for window in windows:
    shifted_prices_window = stock_prices.shift(window, freq='D').add_suffix(f'_{window}_days_ago')
    shifted_stock_prices = pd.merge(shifted_stock_prices, shifted_prices_window, how='left', left_index=True, right_index=True)
    
# Calculate rolling averages for each column and window size
for col in tickers:
    for window in windows:
        new_col_name = f'{col}_rolling_{window}_day_avg'
        stock_prices[new_col_name] = stock_prices[col].rolling(window).mean()




# Combine the original DataFrame, shifted DataFrame, and log prices DataFrame
main_stock_price_df = pd.concat([stock_prices, shifted_stock_prices, stock_price_log], axis=1)

for col in tickers:
    for window in [7, 14, 30, 90]:
        shifted_col = f'{col}_{window}_days_ago'
        new_col_name = f'{col}_pct_change_{window}_days'
        main_stock_price_df[new_col_name] = (main_stock_price_df[col] - main_stock_price_df[shifted_col]) / main_stock_price_df[shifted_col]
        

pd.set_option('display.max_columns', 300)
display(main_stock_price_df.head())

,AAL,UAL,JBLU,OO,ALK,SAVE,LUV,DAL,HA,AAL_rolling_7_day_avg,AAL_rolling_14_day_avg,AAL_rolling_30_day_avg,AAL_rolling_90_day_avg,UAL_rolling_7_day_avg,UAL_rolling_14_day_avg,UAL_rolling_30_day_avg,UAL_rolling_90_day_avg,JBLU_rolling_7_day_avg,JBLU_rolling_14_day_avg,JBLU_rolling_30_day_avg,JBLU_rolling_90_day_avg,ALK_rolling_7_day_avg,ALK_rolling_14_day_avg,ALK_rolling_30_day_avg,ALK_rolling_90_day_avg,SAVE_rolling_7_day_avg,SAVE_rolling_14_day_avg,SAVE_rolling_30_day_avg,SAVE_rolling_90_day_avg,LUV_rolling_7_day_avg,LUV_rolling_14_day_avg,LUV_rolling_30_day_avg,LUV_rolling_90_day_avg,DAL_rolling_7_day_avg,DAL_rolling_14_day_avg,DAL_rolling_30_day_avg,DAL_rolling_90_day_avg,HA_rolling_7_day_avg,HA_rolling_14_day_avg,HA_rolling_30_day_avg,HA_rolling_90_day_avg,AA_last_close,UA_last_close,B6_last_close,OO_last_close,AS_last_close,NK_last_close,WN_last_close,DL_last_close,HA_last_close,AAL_7_days_ago,UAL_7_days_ago,JBLU_7_days_ago,OO_7_days_ago,ALK_7_days_ago,SAVE_7_days_ago,LUV_7_days_ago,DAL_7_days_ago,HA_7_days_ago,AAL_14_days_ago,UAL_14_days_ago,JBLU_14_days_ago,OO_14_days_ago,ALK_14_days_ago,SAVE_14_days_ago,LUV_14_days_ago,DAL_14_days_ago,HA_14_days_ago,AAL_30_days_ago,UAL_30_days_ago,JBLU_30_days_ago,OO_30_days_ago,ALK_30_days_ago,SAVE_30_days_ago,LUV_30_days_ago,DAL_30_days_ago,HA_30_days_ago,AAL_90_days_ago,UAL_90_days_ago,JBLU_90_days_ago,OO_90_days_ago,ALK_90_days_ago,SAVE_90_days_ago,LUV_90_days_ago,DAL_90_days_ago,HA_90_days_ago,AAL_log_price,UAL_log_price,JBLU_log_price,OO_log_price,ALK_log_price,SAVE_log_price,LUV_log_price,DAL_log_price,HA_log_price,AAL_pct_change_7_days,AAL_pct_change_14_days,AAL_pct_change_30_days,AAL_pct_change_90_days,UAL_pct_change_7_days,UAL_pct_change_14_days,UAL_pct_change_30_days,UAL_pct_change_90_days,JBLU_pct_change_7_days,JBLU_pct_change_14_days,JBLU_pct_change_30_days,JBLU_pct_change_90_days,ALK_pct_change_7_days,ALK_pct_change_14_days,ALK_pct_change_30_days,ALK_pct_change_90_days,SAVE_pct_change_7_days,SAVE_pct_change_14_days,SAVE_pct_change_30_days,SAVE_pct_change_90_days,LUV_pct_change_7_days,LUV_pct_change_14_days,LUV_pct_change_30_days,LUV_pct_change_90_days,DAL_pct_change_7_days,DAL_pct_change_14_days,DAL_pct_change_30_days,DAL_pct_change_90_days,HA_pct_change_7_days,HA_pct_change_14_days,HA_pct_change_30_days,HA_pct_change_90_days
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-10-03,37.70,53.25,17.66,27.70,67.09,43.74,39.74,39.98,49.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.629660,3.974998,2.871302,3.321432,4.206035,3.778263,3.682358,3.688379,3.911823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-10-04,37.63,53.73,17.83,27.89,68.71,43.64,39.16,40.15,50.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.70,53.25,17.66,27.70,67.09,43.74,39.74,39.98,49.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.627802,3.983972,2.880882,3.328268,4.229895,3.775974,3.667656,3.692622,3.924544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-10-05,37.87,53.82,17.86,28.06,68.31,43.16,39.10,40.32,50.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.63,53.73,17.83,27.89,68.71,43.64,39.16,40.15,50.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [5]:
# ticker filter function

for ticker in tickers:
    column_filter = [col for col in main_stock_price_df if col.startswith(ticker)]
    df = main_stock_price_df[column_filter]
    df.to_csv(f'{ticker}_stock_data.csv')
    